In [57]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import mysql.connector
from word2number import w2n
import tkinter as tk
from tkinter import *
import speech_recognition as sr 
import torchaudio
import spacy
import dict


In [58]:
frame = tk.Tk()
frame.title("QueryAssist")
frame.geometry('900x600')
  

''

In [59]:

dict1= {
    "<action>":[],
    "<attr_name>":[],
    "<table_name>":[],
    "<condition_name>":[],
    "<condition>":[],
    "<value>":[],
    "<logic>":[]
      }
dictionary1=dict.dictionary11


dictionary3=dict.dictionary3
r = sr.Recognizer()

In [60]:
def AudioRec():
    with sr.Microphone() as source:
                         
            audio = r.record(source, duration = 10)
            
            MyText = r.recognize_google(audio)
            print(MyText)
    return MyText

In [61]:
def printInput():
    word_list=[]
    word_list=tokenize()
    print(word_list)
    tokens_without_sw=[]
    tokens_without_sw=removeStopWords(word_list)
    print(tokens_without_sw)
    string1=[]
    string1=findColumns(tokens_without_sw)
    print(string1)
    string2=[]
    string2=logicConverter(string1)
    print(string2)
    sentence=[]
    sentence=partsOfSpeechTagging(string2)
    print(sentence)
 
    
    action,attr_name,table_name,condition_name,condition,value,logic=findKeyWords(sentence)
    print(action,attr_name,table_name,condition_name,condition,value,logic)
    
    string4=""
    string4=queryGenerator(action,attr_name,table_name,condition_name,condition,value,logic)
    print(string4)
    dataBaseLink(string4)
  
   

In [62]:
def tokenize():
    #input and tokenization moodule
    text = inputtxt.get(1.0, "end-1c")
    if text == "speech": text  = AudioRec()
    text = text.lower()
    word_list = word_tokenize(text)
    return word_list

In [63]:

from textblob import Word

def removeStopWords(word_list):   
    lem=[]
    for r in word_list:
        lem.append(Word(r).lemmatize())
    print("lem2: ",lem)
    
    ignore_list =['the','record','database','table','information','a','are','is','to','marks','mark','of','in','than',"'s",',','.','me','you','us','an','score','scored']
    tokensWithoutStopWords = [word for word in lem if not word in ignore_list]
    return tokensWithoutStopWords



In [64]:
def findColumns(tokens_without_sw):
    global dictionary1
    to_be_append = ""
    location_is = 100
    selected_is = 0
    string1 = []
    counter = 0
    set_word = 0
    for x in tokens_without_sw:
        i = 0
        if x in dictionary1["ssc"]:
            string1.append("ssc")
        elif x in dictionary1["aggregate"]:
            string1.append("aggregate")   
        elif x in dictionary1["name"]:
            string1.append("name") 
        elif x in dictionary1["hsc"]:
            string1.append("hsc")
        elif x in dictionary1["DESC"]:
            string1.append("DESC")
        elif x in dictionary1["ASC"]:
            string1.append("ASC")
        elif x in dictionary1["SELECT"] and selected_is==0:
            string1.append("SELECT")
            selected_is = 1
        elif x in dictionary1["*"]:
            if counter == 0:
                string1.append("*")
                counter = 1
        elif x in dictionary1["WHERE"]:
            string1.append("WHERE")
        elif x in dictionary1["AND"]:
            string1.append("AND")
        elif x in dictionary1["OR"]:
            string1.append("OR")
        elif x in dictionary1["FROM student"]:
            if location_is == 100:
                string1.append("FROM student")
                location_is = i
        elif x in dictionary1["ORDER BY"]:
            string1.append("ORDER BY")    
        elif x in dictionary1["word"]:
            string1.append(str(w2n.word_to_num(x)))
        else:
            string1.append(x)
        i = i+1
    return string1


In [65]:
def logicConverter(string1):
    global dictionary1
    global dictionary3
    string2=[]
    i=0
    countercounter=0
    to_append=""
    len_string1=len(string1)
    
    for x in string1:
        
        if x in dictionary3["<"]:
            
            if string1[i-1]=="not":
                string1[i]=">"
            else:
                string1[i]="<"
        elif x in dictionary3[">"]:
            
            if string1[i-1]=="not":
                string1[i]="<"
            else:
                string1[i]=">"
            
        elif x in dictionary3["="]:
           
            string1[i]="="
            
            
        elif x in dictionary3["* FROM"]:
            
            string1[i]="* FROM"
            
        elif x in dictionary1["AND"]:
            
            countercounter=0
        elif x in dictionary1["OR"]:
            
            countercounter=0   
        
        elif x in dictionary1["<number>"]:
            
            if i < len_string1:
                if countercounter==0:
                    try:
                        if string1[i+1] in dictionary1["*"]:
                            string1[i]=str(int(string1[i+1])+int(x))
                            
                            countercounter=1
                    except:
                        print()
                
        else:
            string1[i]=x
        if to_append!=string1[i] and string1[i]!="":
            to_append = string1[i]
            string2.append(string1[i])
            
        if countercounter==1 :
            string1[i+1]=""
            countercounter=2
            
    
          
        i=i+1
    
    
    return string2
      

In [66]:
def partsOfSpeechTagging(string2):
    
    #POS tagging module
    print("string2")
    sentence=nltk.pos_tag(string2)
    print("sentence")

    return sentence

In [67]:
def findKeyWords(sentence):
    global dict1
    dict= {
        "NN": ["ssc","hsc","name","id","student","*"],
    }
    i=0
    locc=0
    for i, (x1, x2) in enumerate(sentence):
        if x1 == "WHERE":
            locc = i
            break
    
    set_select = 0
    for i, (x1, x2) in enumerate(sentence):
        before_where = i < locc and locc != 0
        after_where = i > locc and locc != 0
        
        if before_where or not locc:
            if x1 in dictionary1["SELECT"]:
                if not set_select:
                    dict1["<action>"].append(x1)
                    set_select = 1
            elif x2 == "NN" and x1 != "FROM student":
                dict1["<attr_name>"].append(x1)
            elif x2 == "VB" and x1 == "aggregate":
                dict1["<attr_name>"].append(x1)
            elif x1 == "FROM student":
                dict1["<table_name>"].append(x1)
            elif x1 in dict["NN"]:
                dict1["<attr_name>"].append(x1)
        elif after_where:
            if x2 == "NN" and x1 != "FROM student":
                dict1["<condition_name>"].append(x1)
            elif x2 == "VB" and x1 == "aggregate":
                dict1["<condition_name>"].append(x1)
            elif x1 in dict["NN"]:
                dict1["<condition_name>"].append(x1)
        
        if x2 == "$" or x1 in dictionary3:
            dict1["<condition>"].append(x1)    
        elif x1 in dictionary1["AND"]:
            dict1["<logic>"].append(x1) 
        elif x1 in dictionary1["OR"]:
            dict1["<logic>"].append(x1)     
        elif x2 == "CD":
            dict1["<value>"].append(x1)
    
    action, attr_name, table_name, condition_name, condition, value, logic = [dict1[key] for key in [
        "<action>",
        "<attr_name>",
        "<table_name>",
        "<condition_name>",
        "<condition>",
        "<value>",
        "<logic>",
    ]]
    
    condition_name = [i for i in condition_name if i]
    
    dict1.clear()
    dict1 = {
        "<action>": [],
        "<attr_name>": [],
        "<table_name>": [],
        "<condition_name>":[],
        "<condition>":[],
        "<value>":[],
        "<logic>":[]
      }
    
    return action,attr_name,table_name,condition_name,condition,value,logic

In [68]:
def queryGenerator(action,attr_name,table_name,condition_name,condition,value,logic):
    
    #SQL Query formation module
    len_condition_name = len(condition_name)    
    len_condition = len(condition)
    len_value = len(value)
    len_logic = len(logic)
    
    
    phase1 = action + attr_name+ table_name
    
    phase2=""
    
    a=len_condition_name
    b=len_condition
    c=len_value
    def maximum(a,b,c):
  
        if (a>=b) and (a>=c):
            largest = a
      
        elif (b >= a) and (b >= c):
            largest = b
        else:
            largest = c
          
        return largest
  
  
    # Driven code 
   
    lenx=maximum(a, b, c)
    
    if lenx==len_condition_name:
        print()
    else:
        
        cond=condition_name[len_condition_name-1]
        
        for x in range(lenx-1):
            condition_name.append(cond)
    len_condition_name=len(condition_name)
    
    for i in range(len_condition_name):
        phase2 = phase2 + " " + condition_name[i]
        
        if i <len_condition:
            phase2 = phase2 + " " + condition[i]
        else :
            phase2 = phase2 + " " + condition[len_condition-1]    
        if i <len_value: 
            phase2 = phase2 + " " + value[i]
        else :
            phase2 = phase2 + " " + value[len_value-1]
        if len_condition_name > len_logic and i < len_logic:
            phase2 = phase2 + " " + logic[i]
        

    
    
    result =""
    for i in phase1:
        result = result + " " +i
    if phase2!="":
        string4=(result +" "+"WHERE"+ phase2)
    else:
        string4=(result)
    return string4

In [69]:
def dataBaseLink(string4): 
    #Database link module
    resultlist=[]
    myconn = mysql.connector.connect(host = "localhost", user = "root",passwd = "root",database = "student1")   
    cur = myconn.cursor()   
    try:  
        cur.execute(string4)        
        result = cur.fetchall()
        resultlist = result        
    except:
        print("error")
        myconn.rollback()       
    myconn.close()
    class Table:
        def __init__(self, root):
            for i in range(total_rows):
                for j in range(total_columns):
                    self.e = Entry(root, width=20, fg='RoyalBlue3', font=('Arial',16,'bold'))
                    self.e.grid(row=i, column=j)
                    self.e.insert(END, resultlist[i][j])
                    self.e.config(highlightbackground="black", highlightcolor="black", highlightthickness=1)

    if resultlist != [] :
        total_rows = len(resultlist)
        total_columns = len(resultlist[0])
       
        root = Tk()
        t = Table(root)
        root.title("Results")
        root.mainloop
        
        lbl.config(text = "SQL Query:  "+string4)
        lbl1.config(text = "")
        lbl2.config(text = "")
        
        
    else :
        print("No Data Found")
        lbl.config(text = "SQL Query:  "+string4 + " ?")
        lbl1.config(text = "No Data Found")
        lbl2.config(text = "Please provide the query more clearly.")
        
        
lbl10 = tk.Label(frame, text = "ENTER INPUT :",font='Helvetica 12 bold', bg='lightblue')
lbl10.pack(pady=20)
inputtxt = tk.Text(frame,
                   height = 3,
                   width = 70,
                   bd=5,
                   font='Helvetica 10 bold',relief='groove',padx = 20,pady = 20,bg='lightgray'
                  )
lbl6 = tk.Label(frame, text = "")
lbl6.pack()  
inputtxt.pack()
lbl4 = tk.Label(frame, text = "")
lbl4.pack()
lbl5 = tk.Label(frame, text = "")
lbl5.pack()
printButton = tk.Button(frame,
                        text = "Submit",
                        bd=5,
                        height = 2,
                        width = 12,
                        relief='raised',
                        bg='lightblue',
                        command = printInput)
printButton.pack()
lbl8 = tk.Label(frame, text = "")
lbl8.pack()  
lbl9 = tk.Label(frame, text = "")
lbl9.pack()
# Label Creation
lbl = tk.Label(frame, text = "",font='Helvetica 12 bold')
lbl.pack()
lbl1 = tk.Label(frame, text = "")
lbl1.pack()
lbl2 = tk.Label(frame, text = "")
lbl2.pack() 
    

In [70]:
frame.mainloop()

['select', 'all', 'students', 'with', 'ssc', 'marks', 'greater', 'than', '30', 'and', 'hsc', 'marks', 'greater', 'than', '60', 'and', 'id', 'greater', 'than', '5']
lem2:  ['select', 'all', 'student', 'with', 'ssc', 'mark', 'greater', 'than', '30', 'and', 'hsc', 'mark', 'greater', 'than', '60', 'and', 'id', 'greater', 'than', '5']
['select', 'all', 'student', 'with', 'ssc', 'greater', '30', 'and', 'hsc', 'greater', '60', 'and', 'id', 'greater', '5']
['SELECT', '*', 'FROM student', 'WHERE', 'ssc', 'greater', '30', 'AND', 'hsc', 'greater', '60', 'AND', 'id', 'greater', '5']

['SELECT', '*', 'FROM student', 'WHERE', 'ssc', '>', '30', 'AND', 'hsc', '>', '60', 'AND', 'id', '>', '5']
string2
sentence
[('SELECT', 'NNP'), ('*', 'NNP'), ('FROM student', 'NNP'), ('WHERE', 'NNP'), ('ssc', 'VBZ'), ('>', '$'), ('30', 'CD'), ('AND', 'NNP'), ('hsc', 'VBD'), ('>', 'NNP'), ('60', 'CD'), ('AND', 'NNP'), ('id', 'NN'), ('>', 'VBD'), ('5', 'CD')]
['SELECT'] ['*'] ['FROM student'] ['ssc', 'hsc', 'id'] ['>', 